In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Attention, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split

In [2]:
X_cents = np.load('dataset/cents.npy')
X_counts = np.load('dataset/counts.npy')
y = np.load('dataset/labels.npy')

In [3]:
print(f'X_cents shape: {X_cents.shape}')
print(f'X_counts shape: {X_counts.shape}')
print(f'y shape: {y.shape}')

X_cents shape: (149084, 1024)
X_counts shape: (149084, 73)
y shape: (149084, 40)


In [4]:
X_cents_train, X_cents_temp, X_counts_train, X_counts_temp, y_train, y_temp = train_test_split(X_cents, X_counts, y, test_size=0.3, random_state=42)
X_cents_val, X_cents_test, X_counts_val, X_counts_test, y_val, y_test = train_test_split(X_cents_temp, X_counts_temp, y_temp, test_size=0.5, random_state=42)

In [5]:
seq_input = Input(shape=(None, 1))
static_input = Input(shape=(X_counts.shape[1],))

In [8]:
# DeepGRU
x = GRU(256, return_sequences=True)(seq_input)
x = GRU(256, return_sequences=True)(x)

In [9]:
# Attention
attention = Attention()([x, x])
context = tf.reduce_sum(attention, axis=1)

In [10]:
x = Dense(256, activation='relu')(context)
#x = Concatenate()([x, static_input])
x = Dense(128, activation='relu')(x)
output = Dense(y.shape[1], activation='softmax')(x)

In [11]:
model = Model(inputs=seq_input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 1)]            0         []                            
                                                                                                  
 gru_1 (GRU)                 (None, None, 256)            198912    ['input_1[0][0]']             
                                                                                                  
 gru_2 (GRU)                 (None, None, 256)            394752    ['gru_1[0][0]']               
                                                                                                  
 attention (Attention)       (None, None, 256)            0         ['gru_2[0][0]',               
                                                                     'gru_2[0][0]']           

In [ ]:
model.fit(X_cents_train, y_train, validation_data=(X_cents_val, y_val), epochs=10)

Epoch 1/10


W0000 00:00:1744232946.647929    4519 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "240" frequency: 3193 num_cores: 16 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 16777216 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


3262/3262 [==============================] - ETA: 0s - loss: 7.3497 - accuracy: 0.0560

W0000 00:00:1744242857.944955    4519 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "240" frequency: 3193 num_cores: 16 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 16777216 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


3262/3262 [==============================] - 10621s 3s/step - loss: 7.3497 - accuracy: 0.0560 - val_loss: 3.5577 - val_accuracy: 0.0587
Epoch 2/10
  23/3262 [..............................] - ETA: 2:40:06 - loss: 3.5489 - accuracy: 0.0543